In [1]:
#import nibabel as nib
import numpy as np
import pandas as pd
import cv2
import glob
import matplotlib.pyplot as plt
from pathlib import Path
import os
import torch
import matplotlib.pyplot as plt
from monai.transforms import (
    LoadImage, EnsureChannelFirst, Orientation, Spacing, ScaleIntensityRange,ScaleIntensityRangePercentiles
)
import pydicom
from tqdm.auto import tqdm

In [6]:



input_path="/ssd3/rsna_2025_flayer/input"
folder = "/ssd3/rsna_2025_flayer/data/segmentations"
out_folder = "spacing_07_p1_p99_v1"

In [3]:
df = pd.read_csv(f"{input_path}/train_with_folds_optimized_axis_v1.csv")

In [5]:
df=df[df["axis"]=="z"].reset_index(drop=True)

In [7]:


files = glob.glob(f"{folder}/*.nii")
files = [f for f in files if not f.endswith("cowseg.nii")]
nii_sid = [Path(f).stem for f in files]  

In [8]:
len(nii_sid)

178

In [9]:
nii_sid = [sid for sid in nii_sid if sid in df["SeriesInstanceUID"].unique()]

In [10]:
len(nii_sid)

166

In [11]:
#nii_sid

In [12]:
ct_nii_sids=[sid for sid in nii_sid if sid in df[df["Modality"]=="CTA"]["SeriesInstanceUID"].unique()]
mri_nii_sids=[sid for sid in nii_sid if sid in df[df["Modality"]!="CTA"]["SeriesInstanceUID"].unique()]

In [13]:

from monai.transforms import (
    Compose, EnsureChannelFirst, Orientation, Spacing,
    ScaleIntensityRangePercentiles
)
from monai.transforms import LoadImage
import numpy as np
import torch
from typing import Tuple, Optional, Dict

# ---- 共同工具 ----
def _to_DHW_rot90(t: torch.Tensor, k: int = 1):
    arr = t.numpy() if isinstance(t, torch.Tensor) else t
    arr = np.squeeze(arr, axis=0)        # [H,W,D]
    arr = np.transpose(arr, (2, 0, 1))   # -> [D,H,W]
    arr = np.rot90(arr, k=k, axes=(1, 2))
    return arr

def _clean_mask_labels(msk: np.ndarray) -> np.ndarray:
    msk = (msk > 0).astype(np.uint8)
    return msk.astype(np.uint8)

# ---- 轉換（MRI 影像 / Mask）----



# MRI 預處理
_img_t_mri = Compose([
    EnsureChannelFirst(),
    Orientation(axcodes="RAS"),
    Spacing(pixdim=(0.7, 0.7, 0.7), mode="bilinear"),
    # ScaleIntensityRangePercentiles(
    #     lower=0.5, upper=99.5, b_min=0.0, b_max=1.0, clip=True
    # ),
    ScaleIntensityRangePercentiles(
        lower=1.0, upper=99.0, b_min=0.0, b_max=1.0, clip=True
    ),
])

# CT 預處理
_img_t_ct = Compose([
    EnsureChannelFirst(),
    Orientation(axcodes="RAS"),
    Spacing(pixdim=(0.7, 0.7, 0.7), mode="bilinear"),
    # ScaleIntensityRange(
    #     a_min=-1000, a_max=2000, b_min=0.0, b_max=1.0, clip=True
    # ),
    ScaleIntensityRangePercentiles(
        lower=1.0, upper=99.0, b_min=0.0, b_max=1.0, clip=True
    ),
])

_msk_t = Compose([
    EnsureChannelFirst(),
    Orientation(axcodes="RAS"),
    Spacing(pixdim=(0.7, 0.7, 0.7), mode="nearest"),
])

# ---- 主函式 ----
def process_mri_case(
    sid: str,
    modality: str = "mri",   # "mri" 或 "ct"
    img_path_tpl: str = "/ssd3/rsna_2025_flayer/data/segmentations/{sid}.nii",  # 影像 NIfTI 路徑樣板
    msk_path_tpl: Optional[str] = "/ssd3/rsna_2025_flayer/data/segmentations/{sid}_cowseg.nii",# 標註 NIfTI 路徑樣板
    rot_k: int = 1,
    return_meta: bool = False,
) -> Tuple[np.ndarray, Optional[np.ndarray], Optional[Dict]]:
    """
    回傳:
      img_arr: (D,H,W) float32, 已做強度標準化
      msk_arr: (D,H,W) uint8, 已將 >13 壓為 1；若無 mask，回傳 None
      info:    可選，包含 affine 等資訊
    """
    loader = LoadImage(image_only=False)

    # --- 影像 ---
    #print(sid)
    img_p = img_path_tpl.format(sid=sid)
    img, img_meta = loader(img_p)            # img: torch.Tensor or np

    
    if modality.lower() == "ct":
        img = _img_t_ct(img)
    else:  # 預設 mri
        img = _img_t_mri(img)
        
    img_arr = _to_DHW_rot90(img, k=rot_k).astype(np.float32)
    # 保險處理 NaN/Inf
    img_arr = np.nan_to_num(img_arr, nan=0.0, posinf=0.0, neginf=0.0)

    # --- 標註（可選） ---
    msk_arr = None
    if msk_path_tpl is not None:
        msk_p = msk_path_tpl.format(sid=sid)
        try:
            msk, msk_meta = loader(msk_p)
            msk = _msk_t(msk)
            msk_arr = _to_DHW_rot90(msk, k=rot_k) 
            msk_arr_b = _clean_mask_labels(msk_arr)  # >13 → 1，且轉 uint8
        except FileNotFoundError:
            msk_meta = None
            msk_arr = None
            msk_arr_b = None

    if not return_meta:
        return img_arr, msk_arr_b, msk_arr, None

    info = {
        "sid": sid,
        "img_path": img_p,
        "msk_path": (msk_path_tpl.format(sid=sid) if msk_path_tpl else None),
        "img_affine": img_meta.get("affine", None) if isinstance(img_meta, dict) else None,
        "img_original_affine": img_meta.get("original_affine", None) if isinstance(img_meta, dict) else None,
        "msk_affine": (msk_meta.get("affine", None) if (msk_arr is not None and isinstance(msk_meta, dict)) else None),
        "msk_original_affine": (msk_meta.get("original_affine", None) if (msk_arr is not None and isinstance(msk_meta, dict)) else None),
        "rot_k": rot_k,
        "spacing_mm": (0.7, 0.7, 0.7),
        "orientation": "RAS",
    }
    return img_arr, msk_arr_b,msk_arr, info



/home/fateplsf/miniconda3/envs/py3.10/lib/python3.10/site-packages/monai/utils/deprecate_utils.py:321: FutureWarning: monai.transforms.spatial.array Orientation.__init__:labels: Current default value of argument `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` was changed in version None from `labels=(('L', 'R'), ('P', 'A'), ('I', 'S'))` to `labels=None`. Default value changed to None meaning that the transform now uses the 'space' of a meta-tensor, if applicable, to determine appropriate axis labels.
  warn_deprecated(argname, msg, warning_category)


In [ ]:

os.makedirs(f"{input_path}/{out_folder}",exist_ok=True)

os.makedirs(f"{input_path}/{out_folder}/img",exist_ok=True)
os.makedirs(f"{input_path}/{out_folder}/mask_b",exist_ok=True)
#os.makedirs(f"{input_path}/{out_folder}/mask",exist_ok=True)


In [15]:
from tqdm.auto import tqdm

In [ ]:
for sid in tqdm(ct_nii_sids):
    img_arr,msk_arr_b,msk_arr,_=process_mri_case(sid,"ct")
    np.save(f"{input_path}/{out_folder}/img/{sid}.npy",img_arr)
    np.save(f"{input_path}/{out_folder}/mask_b/{sid}.npy",msk_arr_b)
    #np.save(f"{input_path}/{out_folder}/mask/{sid}.npy",msk_arr)

for sid in tqdm(mri_nii_sids):
    img_arr,msk_arr_b,msk_arr,_=process_mri_case(sid,"mri")
    np.save(f"{input_path}/{out_folder}/img/{sid}.npy",img_arr)
    np.save(f"{input_path}/{out_folder}/mask_b/{sid}.npy",msk_arr_b)
    #np.save(f"{input_path}/{out_folder}/mask/{sid}.npy",msk_arr)


  0%|          | 0/73 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

In [34]:
#spacing_07 0.7mm ,mri:ScaleIntensityRangePercentiles(lower=0.5, upper=99.5),  ct:ScaleIntensityRange(a_min=-1000, a_max=2000)
#spacing_07_p1_p99 ScaleIntensityRangePercentiles(lower=1.0, upper=99.0) 